Das Problem:
Die Json Dateien sind nicht alle gleich.

Wenn dein Program also auf eine Datei stösst, die eine unerwartete Struktur hat, stürzt es ab, obwohl es für korrekte dateien vermutlich gut funktionieren wird.

z.B. ich schaue explizit nach einer Column "categories". Manche der JSON Dateien haben das aber nicht.
Das führt dann zum Fehler.

Die Teil-Lösung, ist - wir programmieren das defensiv, so dass nur explizit "gute" Dateien verarbetier werden.
Wir schauen also nach, ob die datei die Struktur hat, die wir auch erwarten.

    if "categories" in df.columns:
        ...

Wenn ja, verarbeiten wir die Datei; Wenn nein überspringen wir sie.

Das vermeidet den Programfehler aber wir verlieren einen Teil der Daten.

Ich habe unten einen Code geschrieben, der die "guten" Dateien verarbeitet (das sind 1226 von 1609, also so ca 75% - man könnte zu Not sagen, dass es reicht)

Zusätzlich macht das Program eine Liste von Dateien die nicht verarbeitet werden konnten - d.h. man kann sich die anschauen, und zusätzliche methoden schreiben, die diese anderen Dateien verarbeiten können (wiederum - defensiv - die methode wird nur aufgerufen, wenn die Datei dazu passt.)

Auf diese weise sollten mit jeder neuen Methode weniger "schlechte" Dateien übersprungen werden.

### pandas read_json() demo

In [1]:
import pandas as pd

df = pd.read_json("test_file.json")

df.head()

# we will for now read one json file as a dataframe.
# infortunately the file is nested
# {
#    categories {
#        name       # cat_name
#        category   # cat_desc
#        items[     # list of items per category
#            {
#               name    # item properties
#               price
#               ingredients   
#               description
#            },
#            ......    more items
#        ]
#    },
#    ......    more categories
# }
# so we got a dataftame of categories, each holding a list of items.
# below we will unpack the data such that each item is a row in the final dataframe

,categories
0,"{'name': 'Bowls', 'category': 'Bowls', 'items'..."
1,"{'name': 'Salate', 'category': 'Alle Salate we..."
2,"{'name': 'Burger', 'category': 'Alle Burger we..."
3,"{'name': 'Burger Menü', 'category': '', 'items..."
4,"{'name': 'Pizza, Ø 30cm', 'category': 'Alle Pi..."


In [2]:
category_info = pd.json_normalize(df["categories"])

category_info.head()

# here we unfolded the dataframe extracting category name and description
# the items are still a list that we need to further unpack

,name,category,items
0,Bowls,Bowls,"[{'name': 'Chicken Bowl', 'price': '13,90 €', ..."
1,Salate,"Alle Salate werden mit buntem Salat, Gurken, T...","[{'name': 'Falafel Salat', 'price': '10,90 €',..."
2,Burger,Alle Burger werden mit 100% Rindfleisch (200g)...,"[{'name': 'Classic Beef Hamburger', 'price': '..."
3,Burger Menü,,"[{'name': 'Classic Beef Hamburger Menü', 'pric..."
4,"Pizza, Ø 30cm",Alle Pizzen werden mit Tomatensauce und Käse z...,"[{'name': 'Pizza Margherita', 'price': '8,90 €..."


In [3]:
# this is an example single row just to see how the data is stored
# row is a dict with name, category and items

row = category_info.iloc[1]

print(row)

cat_name = row["name"]
cat_desc = row["category"]

print()
print(f"{cat_name=}")
print(f"{cat_desc=}")
print(f"{row['items']=}")

name                                                   Salate
category    Alle Salate werden mit buntem Salat, Gurken, T...
items       [{'name': 'Falafel Salat', 'price': '10,90 €',...
Name: 1, dtype: object

cat_name='Salate'
cat_desc='Alle Salate werden mit buntem Salat, Gurken, Tomaten, Paprika, Mais und roten Zwiebeln zubereitet.'
row['items']=[{'name': 'Falafel Salat', 'price': '10,90\xa0€', 'ingredients': 'Falafel, gemischte Wildkräuter und Blattsalat, Tomaten, Gurken,Avocado & Brot', 'description': 'Wahl aus: mit Balsamico-Dressing, mit Dressing, hausgemacht, mit Honig-Senf-Dressing oder ohne Dressing.'}, {'name': 'Gemischter Salat mit Hähnchenbruststreifen (groß)', 'price': '11,90\xa0€', 'ingredients': 'Hähnchenbrust, gemischte Wildkräuter und Blattsalat, Avocado, Parmesan, Tomaten, Gurken,Paprika, Nussmix und Brot', 'description': 'Wahl aus: mit Balsamico-Dressing, mit Dressing, hausgemacht, mit Honig-Senf-Dressing oder ohne Dressing.'}, {'name': 'Beilagensalat (klein)', 'pri

In [4]:
# now we will iterate through all categories and through each item in each category
# to fully "flatten" the nested dataframe extracting each item as it's own row

all_items = []

# iterating categories
for _, row in category_info.iterrows():
    cat_name = row["name"]
    cat_desc = row["category"]

    # iterating items within one category
    for i in row["items"]:
        # i will be a dict with name, price, ingredients and description of an item
        # we also add the category info to each item
        i["category_name"] = cat_name
        i["category_text"] = cat_desc

        all_items.append(i)

# create dataframe from the items list
full_items_df = pd.DataFrame(all_items)

print(full_items_df.info())
print("\ntake a look at a single row:")
row = full_items_df.iloc[1]
print(row)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           32 non-null     object
 1   price          32 non-null     object
 2   ingredients    32 non-null     object
 3   description    32 non-null     object
 4   category_name  32 non-null     object
 5   category_text  32 non-null     object
dtypes: object(6)
memory usage: 1.6+ KB
None

take a look at a single row:
name                                                  Falafel Bowl
price                                                      13,90 €
ingredients      Wildreis, Falafel, Salat, gegrillte Paprika, F...
description                                                       
category_name                                                Bowls
category_text                                                Bowls
Name: 1, dtype: object


In [5]:
# we processed a single file into a dataframe.
# we will convert this into a function, that we can successively call on each meny file

### Attempt to read a diffrent JSON schema
    [
        {
            "model": "m1",
            "section_name": "",
            "items": [
                {
                    "name": "Insalata Verde",
                    "description": "",
                    "ingredients": [
                        "Eisbergsalat",
                        " Gurken",
                        " Mais",
                        " Rucola"
                    ],
                    "price_raw": "3,00 €",
                    "price": 3.0
                },

In [6]:
df = pd.read_json("test_file_pattern_2.json")

In [7]:
df.head()

,model,section_name,items
0,m1,,"[{'name': 'Insalata Verde', 'description': '',..."


In [8]:
all_items = []

for _, row in df.iterrows():
    cat_name = row["model"]
    cat_desc = row["section_name"]

    for i in row["items"]:
        i["category_name"] = cat_name
        i["category_text"] = cat_desc
        i["price"] = i["price_raw"]
        i.pop("price_raw", None)

        all_items.append(i)

full_df = pd.DataFrame(all_items)
print(full_df.head())

                   name description  \
0        Insalata Verde               
1        Insalata Verde               
2  Insalata di Pomodori               
3        Insalata Mista               
4     Insalata Italiana               

                                         ingredients   price category_name  \
0            [Eisbergsalat,  Gurken,  Mais,  Rucola]  3,00 €            m1   
1            [Eisbergsalat,  Gurken,  Mais,  Rucola]  4,00 €            m1   
2                               [Tomaten,  Zwiebeln]  4,00 €            m1   
3                          [gemischter Salat,  Mais]  6,00 €            m1   
4  [gemischter Salat,  Mais,  Zwiebeln,  Schinken...  6,50 €            m1   

  category_text  
0                
1                
2                
3                
4                
